In [5]:
import numpy as np

In [3]:
numpy.__version__

'1.10.1'

In [10]:
kek1 = np.random.normal(size=(100,100))

In [11]:
kek1

array([[ 1.14182382, -0.22084166, -0.23341984, ...,  0.28923463,
        -0.441956  ,  0.44742408],
       [ 0.51286824, -0.68243561, -0.17657342, ...,  0.4964649 ,
         0.51332145, -0.88067166],
       [-0.18394034,  1.04993397, -0.00750562, ...,  0.36660801,
         0.3168466 , -0.83866734],
       ..., 
       [-0.01749497,  1.00991166, -0.79460783, ..., -1.04397142,
         0.98493422,  1.08621583],
       [ 0.5591931 ,  0.40860439, -1.08736397, ..., -0.55515468,
         1.23748906, -1.28385881],
       [ 0.60461176,  0.65733973,  0.25329209, ..., -0.88755962,
         0.84842241,  0.86195479]])

In [12]:
kek2 = np.random.normal(size=(100,100))

In [13]:
import time

In [18]:
dum=np.argmax(kek2,axis=0)

In [22]:
timeit kek2[np.arange(100),dum]

The slowest run took 30.92 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 4.57 µs per loop


In [23]:
import pandas as pd

In [24]:
data = pd.read_csv('yeast_example_10.txt', header=None, sep='\t')

In [25]:
import scipy.stats as sps

In [26]:
data[:][:] = sps.mstats.zscore(data,axis=1,ddof=1)

In [27]:
import argparse

In [28]:
def parse_args():
	parser = argparse.ArgumentParser()
	parser.add_argument('--Input', dest='input', type=argparse.FileType('r'), required=True, help='CSV expression file. Gene names in first column, time points in first row. Single profile (average your replicates).')
	parser.add_argument('--Pool', dest='pool', type=int, default=1, help='Number of processes to use when precomputing the Qian matrix and later on in AP clustering. Default: 1 (no parallelisation)')
	parser.add_argument('--Iterations', dest='iters', type=int, default=1000, help='Number of Affinity Propagation steps to take in the clustering. Default: 1000')
	parser.add_argument('--Converge', dest='conv', type=int, default=1000, help='If the cluster affinity of all genes remains unchanged for this many steps, the clustering is deemed completed ahead of time. Default: 100')
	parser.add_argument('--Lambda', dest='damp', type=float, default=0.9, help='Dampening of the A and R matrix values to prevent numerical oscillations. Default: 0.9')
	parser.add_argument('--SelfSim', dest='s', type=float, default=0, help='Self similarity score override in the Qian similarity matrix to prevent degenerate cases. If 0, then set to median of matrix. Default: 0')
	args = parser.parse_args(['--Input','yeast_example_10.txt'])
	return args

In [31]:
import scipy.spatial as spsp

In [34]:
spsp.distance.squareform(np.arange(10)+1)

array([[  0.,   1.,   2.,   3.,   4.],
       [  1.,   0.,   5.,   6.,   7.],
       [  2.,   5.,   0.,   8.,   9.],
       [  3.,   6.,   8.,   0.,  10.],
       [  4.,   7.,   9.,  10.,   0.]])

In [35]:
def qian(X,Y):
	'''A function that computes the AP-Qian similarity measure between two expression profiles.'''
	holderplus = np.zeros((len(X)+1,len(X)+1))
	holderminus = np.zeros((len(X)+1,len(X)+1))
	for i in np.arange(1,len(X)+1):
		for j in np.arange(1,len(X)+1):
			holderplus[i,j] = np.max([0, holderplus[i-1,j-1] + X[i-1]*Y[j-1]])
			holderminus[i,j] = np.max([0, holderminus[i-1,j-1] - X[i-1]*Y[j-1]])
	return np.max([np.max(holderplus),np.max(holderminus)])-len(X)+1

In [36]:
import time

In [74]:
dum1 = np.arange(10)
t0 = time.clock()
for i in np.arange(1000):
    qian(dum1,dum1)
t1 = time.clock()

t1-t0

0.4376039999999932

In [46]:
def qian(X,Y):
	'''A function that computes the AP-Qian similarity measure between two expression profiles.'''
	holderplus = np.zeros((len(X)+1,len(X)+1))
	holderminus = np.zeros((len(X)+1,len(X)+1))
	for i in np.arange(1,len(X)+1):
		for j in np.arange(1,len(X)+1):
			holderplus[i,j] = np.amax([0, holderplus[i-1,j-1] + X[i-1]*Y[j-1]])
			holderminus[i,j] = np.amax([0, holderminus[i-1,j-1] - X[i-1]*Y[j-1]])
	return np.amax([np.amax(holderplus),np.amax(holderminus)])-len(X)+1

In [49]:
kek2

array([[ 0.70148846,  0.03780897,  0.26311262, ..., -0.26531249,
        -0.30894105, -0.73745715],
       [-0.72347091,  1.03709728, -0.19456248, ..., -0.67930094,
        -0.70475245,  0.74104262],
       [ 1.05485814, -0.76117091, -0.35924036, ...,  1.86004076,
         0.16851401,  0.25344436],
       ..., 
       [ 1.26137745, -0.32896052, -0.71110071, ...,  0.43522263,
        -0.52499464, -0.78803389],
       [-0.05566309,  0.47851969, -1.04937906, ..., -2.25214771,
        -2.15761118, -0.46943753],
       [ 0.18192434,  0.14083539, -2.17694165, ..., -0.7924008 ,
        -0.8601154 ,  0.84218599]])

In [52]:
import functools

In [53]:
timeit functools.reduce(np.maximum, kek2)

10000 loops, best of 3: 110 µs per loop


In [54]:
timeit np.max(kek2)

The slowest run took 16.40 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 8.77 µs per loop


In [68]:
def qian(X,Y):
	'''A function that computes the AP-Qian similarity measure between two expression profiles.'''
	holderplus = np.zeros((len(X)+1,len(X)+1))
	holderminus = np.zeros((len(X)+1,len(X)+1))
	out = 0
	for i in np.arange(1,len(X)+1):
		for j in np.arange(1,len(X)+1):
			#don't ask. this is faster.
			h1 = holderplus[i-1,j-1] + X[i-1]*Y[j-1]
			h2 = holderminus[i-1,j-1] - X[i-1]*Y[j-1]
			if h1>0:
				holderplus[i,j] = h1
				if h1>out:
					out=h1
			if h2>0:
				holderminus[i,j] = h2
				if h2>out:
					out=h2
	return out-len(X)+1

In [75]:
%load_ext line_profiler
%lprun -f function_to_profile statement_that_invokes_the_fuction

UsageError: Could not find function 'function_to_profile'.
NameError: name 'function_to_profile' is not defined

In [77]:
lprun -f qian qian(dum1,dum1)

In [81]:
def qian(X,Y):
	'''A function that computes the AP-Qian similarity measure between two expression profiles.'''
	holderplus = np.zeros((len(X)+1,len(X)+1))
	holderminus = np.zeros((len(X)+1,len(X)+1))
	out = 0
	for i in np.arange(1,len(X)+1):
		for j in np.arange(1,len(X)+1):
			#don't ask. this is faster.
			h1 = holderplus[i-1,j-1] + X[i-1]*Y[j-1]
			h2 = holderminus[i-1,j-1] - X[i-1]*Y[j-1]
			if h1>0:
				holderplus[i,j] = h1
				if h1>out:
					out=h1
			if h2>0:
				holderminus[i,j] = h2
				if h2>out:
					out=h2
	return out-len(X)+1

In [82]:
timeit qian(dum1,dum1)

1000 loops, best of 3: 404 µs per loop


In [85]:
np.dot(np.arange(3),np.transpose(np.arange(3)))

5

In [86]:
np.arange(3)

array([0, 1, 2])

In [87]:
np.transpose(np.arange(3))

array([0, 1, 2])

In [88]:
kik = np.arange(3)

In [93]:
kik.shape = (3,1)

kok = np.arange(3)
kok.shape = (1,3)

In [95]:
np.dot(kik+1,kok+1)

array([[1, 2, 3],
       [2, 4, 6],
       [3, 6, 9]])

In [101]:
def row_index(i, M):
    ii = M*(M+1)/2-1-i
    K = np.floor((np.sqrt(8*ii+1)-1)/2)
    return M-1-K

def column_index(i, M):
    ii = M(M+1)/2-1-i
    K = np.floor((np.sqrt(8*ii+1)-1)/2)
    return i - M*(M+1)/2 + (K+1)*(K+2)/2

In [102]:
row_index(9,4)

3.0

In [103]:
kek = np.arange(9)
kek.shape = (3,3)

In [104]:
kek

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [106]:
np.take(kek,3)

3

In [107]:
kek2

array([[ 0.70148846,  0.03780897,  0.26311262, ..., -0.26531249,
        -0.30894105, -0.73745715],
       [-0.72347091,  1.03709728, -0.19456248, ..., -0.67930094,
        -0.70475245,  0.74104262],
       [ 1.05485814, -0.76117091, -0.35924036, ...,  1.86004076,
         0.16851401,  0.25344436],
       ..., 
       [ 1.26137745, -0.32896052, -0.71110071, ...,  0.43522263,
        -0.52499464, -0.78803389],
       [-0.05566309,  0.47851969, -1.04937906, ..., -2.25214771,
        -2.15761118, -0.46943753],
       [ 0.18192434,  0.14083539, -2.17694165, ..., -0.7924008 ,
        -0.8601154 ,  0.84218599]])

In [108]:
timeit kek2[77,77]

The slowest run took 34.77 times longer than the fastest. This could mean that an intermediate result is being cached 
10000000 loops, best of 3: 148 ns per loop


In [109]:
timeit np.take(kek2,77*77)

The slowest run took 15.12 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 2.12 µs per loop


In [116]:
def qian(X,Y):
	'''A function that computes the AP-Qian similarity measure between two expression profiles.'''
	holderplus = np.zeros((len(X)+1,len(X)+1))
	holderminus = np.zeros((len(X)+1,len(X)+1))
	out = 0
	for i in np.arange(1,len(X)+1):
		for j in np.arange(1,len(X)+1):
			#don't ask. this is faster.
			h1 = holderplus[i-1,j-1] + X[i-1]*Y[j-1]
			h2 = holderminus[i-1,j-1] - X[i-1]*Y[j-1]
			if h1>0:
				holderplus[i,j] = h1
				if h1>out:
					out=h1
			if h2>0:
				holderminus[i,j] = h2
				if h2>out:
					out=h2
	return out-len(X)+1

In [111]:
timeit qian(dum1,dum1)

1000 loops, best of 3: 416 µs per loop


In [119]:
def qian(X,Y):
	'''A function that computes the AP-Qian similarity measure between two expression profiles.'''
	holderplus = np.zeros((len(X)+1,len(X)+1))
	holderminus = np.zeros((len(X)+1,len(X)+1))
	out = 0
	for i in np.arange(1,len(X)+1):
		for j in np.arange(1,len(X)+1):
			#don't ask. this is faster.
			h0 = np.take(X,i-1)*np.take(Y,j-1)
			h1 = holderplus[i-1,j-1] + h0
			h2 = holderminus[i-1,j-1] - h0
			if h1>0:
				holderplus[i,j] = h1
				if h1>out:
					out=h1
			if h2>0:
				holderminus[i,j] = h2
				if h2>out:
					out=h2
	return out-len(X)+1

In [113]:
timeit qian(dum1,dum1)

1000 loops, best of 3: 825 µs per loop


In [117]:
dum1 = np.arange(20)
t0 = time.clock()
for i in np.arange(1000):
    qian(dum1,dum1)
t1 = time.clock()

t1-t0

1.6362290000000002

In [120]:
lprun -f qian qian(dum1,dum1)

In [121]:
args

NameError: name 'args' is not defined

In [122]:
args = parse_args()

In [123]:
args

Namespace(conv=1000, damp=0.9, input=<_io.TextIOWrapper name='yeast_example_10.txt' mode='r' encoding='UTF-8'>, iters=1000, pool=1, s=0)

In [126]:
def single_psistar(i, psistar, data):
	for j in np.arange(i+1, data.shape[0]):
		val = qian(data[i,:], data[j,:])
		psistar[i,j] = val
		psistar[j,i] = val
	return psistar

def get_psistar(data, args):
	'''Computes the AP-Qian similarity matrix'''
	t0 = time.clock()
	if args.pool > 1:
		psistar_base = mp.Array(ctypes.c_double, data.shape[0]*(data.shape[0]-1)/2)
		psistar = np.ctypeslib.as_array(psistar_base.get_obj())
	else:
		psistar = np.zeros(data.shape[0],data.shape[0])
	for i in np.arange(data.shape[0]-1):
		psistar = single_psistar(i, psistar, data)

In [130]:
timeit get_psistar(data, args)

TypeError: data type not understood

In [132]:
get_psistar(data,args)

TypeError: data type not understood

In [133]:
def single_psistar(i, psistar, data):
	for j in np.arange(i+1, data.shape[0]):
		val = qian(data[i,:], data[j,:])
		psistar[i,j] = val
		psistar[j,i] = val
	return psistar

def get_psistar(data, args):
	'''Computes the AP-Qian similarity matrix'''
	t0 = time.clock()
	if args.pool > 1:
		psistar_base = mp.Array(ctypes.c_double, data.shape[0]*(data.shape[0]-1)/2)
		psistar = np.ctypeslib.as_array(psistar_base.get_obj())
	else:
		#psistar_vector = np.zeros(data.shape[0]*(data.shape[0]-1)/2)
		psistar = np.zeros((data.shape[0],data.shape[0]))
	for i in np.arange(data.shape[0]-1):
		psistar = single_psistar(i, psistar, data)

In [134]:
timeit get_psistar(data,args)

10 loops, best of 3: 138 ms per loop


In [135]:
import numpy as np
import scipy.stats as spstats
import scipy.spatial as spspatial
import pandas as pd
import argparse
import multiprocessing as mp
import ctypes
import sys

In [136]:
def get_psistar_vec(data, args):
	'''Computes the AP-Qian similarity matrix'''
	t0 = time.clock()
	if args.pool > 1:
		psistar_base = mp.Array(ctypes.c_double, data.shape[0]*(data.shape[0]-1)/2)
		psistar = np.ctypeslib.as_array(psistar_base.get_obj())
	else:
		psistar_vector = np.zeros(data.shape[0]*(data.shape[0]-1)/2)
		#psistar = np.zeros((data.shape[0],data.shape[0]))
	k=0
	for i in np.arange(data.shape[0]-1):
		#psistar = single_psistar(i, psistar, data)
		for j in np.arange(i+1,data.shape[0]):
			val = qian(data[i,:],data[j,:])
			psistar_vector[k] = val
			k=k+1
	psistar = spspatial.distance.squareform(psistar_vector)

In [137]:
timeit get_psistar_vec(data,args)

10 loops, best of 3: 134 ms per loop


//anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:8: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [138]:
def get_psistar(data, args):
	'''Computes the AP-Qian similarity matrix'''
	t0 = time.clock()
	if args.pool > 1:
		psistar_base = mp.Array(ctypes.c_double, data.shape[0]*data.shape[0])
		psistar = np.ctypeslib.as_array(psistar_base.get_obj())
		psistar = psistar.reshape(input.shape[0], input.shape[0])
	else:
		psistar = np.zeros((data.shape[0],data.shape[0]))
	k=0
	for i in np.arange(data.shape[0]-1):
		single_psistar(i, psistar, data)

In [139]:
args.pool = 2

In [140]:
timeit get_psistar(data,args)

AttributeError: 'function' object has no attribute 'shape'

In [141]:
def get_psistar(data, args):
	'''Computes the AP-Qian similarity matrix'''
	t0 = time.clock()
	if args.pool > 1:
		psistar_base = mp.Array(ctypes.c_double, data.shape[0]*data.shape[0])
		psistar = np.ctypeslib.as_array(psistar_base.get_obj())
		psistar = psistar.reshape(data.shape[0], data.shape[0])
	else:
		psistar = np.zeros((data.shape[0],data.shape[0]))
	k=0
	for i in np.arange(data.shape[0]-1):
		single_psistar(i, psistar, data)

In [142]:
timeit get_psistar(data,args)

10 loops, best of 3: 131 ms per loop


In [143]:
def get_psistar(data, args):
	'''Computes the AP-Qian similarity matrix'''
	t0 = time.clock()
	if args.pool > 1:
		psistar_base = mp.Array(ctypes.c_double, data.shape[0]*data.shape[0])
		psistar = np.ctypeslib.as_array(psistar_base.get_obj())
		psistar = psistar.reshape(data.shape[0], data.shape[0])
	else:
		psistar = np.zeros((data.shape[0],data.shape[0]))
	k=0
	for i in np.arange(data.shape[0]-1):
		single_psistar(i, psistar, data)
	print(psistar)

In [144]:
get_psistar(data,args)

[[  0.          -9.96621859  -8.17416598  -9.20923155  -9.54997823
   -8.43338573  -7.83480044  -5.50684859  -5.85177818 -10.21868884]
 [ -9.96621859   0.          -9.14159602  -5.92859693  -6.73902199
   -7.97018475  -9.39384304  -7.8235198   -8.90605239  -5.53412437]
 [ -8.17416598  -9.14159602   0.          -4.77676397  -7.17530533
   -6.69674174  -9.0890633   -7.97131334  -9.41253147  -7.39927648]
 [ -9.20923155  -5.92859693  -4.77676397   0.          -7.40891915
   -4.19955487  -8.87038497  -6.7263422   -7.63663341  -2.12311401]
 [ -9.54997823  -6.73902199  -7.17530533  -7.40891915   0.          -8.39919639
   -9.98245881  -6.91365189  -9.26618426  -7.113631  ]
 [ -8.43338573  -7.97018475  -6.69674174  -4.19955487  -8.39919639   0.
   -8.32079689  -5.98041839 -10.17309229  -8.31888167]
 [ -7.83480044  -9.39384304  -9.0890633   -8.87038497  -9.98245881
   -8.32079689   0.          -5.27046157  -7.02615571 -10.09666837]
 [ -5.50684859  -7.8235198   -7.97131334  -6.7263422   -6.91365

In [145]:
data

array([[ -4.58531963e-01,   4.21290789e-01,   4.42238949e-02,
          2.53705503e-01,   1.69912860e-01,  -1.08697679e+00,
          1.69912860e-01,   7.98357682e-01,  -9.19391500e-01,
         -8.35598857e-01,   5.05083432e-01,   8.82150325e-01,
         -1.08697679e+00,  -1.25456207e+00,   9.24046647e-01,
          2.43231422e+00,  -1.50594000e+00,   5.46979753e-01],
       [ -4.00214640e-01,  -2.04269552e+00,   2.91356258e-01,
         -9.18892814e-01,  -1.14941645e+00,  -1.66809462e+00,
          3.20171712e-01,   6.08326253e-01,   2.62540804e-01,
          1.64568260e+00,  -4.00214640e-01,   8.38849886e-01,
          3.20171712e-02,   1.18463534e-01,   1.47278988e-01,
          1.58805169e+00,  -2.27321916e-01,   9.54111703e-01],
       [ -6.16524428e-02,  -1.60856828e+00,  -3.64309889e-01,
          4.09148030e-01,  -9.52810480e-02,   6.44548266e-01,
          4.42776635e-01,   2.56137876e+00,   2.41005004e-01,
          1.40119188e-01,  -1.96166864e-01,   1.01446292e+00,
      

In [151]:
def single_psistar(i, psistar, input):
	'''Computes a single row of the psistar thing.'''
	for j in np.arange(input.shape[0]):
		if i==j:
			#self-similarity. will be dealt with later
			continue
		psistar[i,j] = qian(input[i,:], input[j,:])
	return psistar

def pool_single_psistar(i):
	'''The same as above, wrapped in a parallel-happy manner'''
	#no need for a return as this is on a shared memory array
	single_psistar(i, psistar, input)

def _psistar_pool_init(_psistar, _input):
	global psistar, input
	psistar = _psistar
	input = _input

def get_psistar(input, args):
	'''A function that computes the psistar similarity matrix using the Qian measure.'''
	if args.pool > 1:
		#shared memory life
		psistar_base = mp.Array(ctypes.c_double, input.shape[0]*input.shape[0])
		psistar = np.ctypeslib.as_array(psistar_base.get_obj())
		psistar = psistar.reshape(input.shape[0], input.shape[0])
		p = mp.Pool(args.pool, _psistar_pool_init, (psistar, input))
		p.map(pool_single_psistar, np.arange(input.shape[0]))
	else:
		psistar = np.zeros((input.shape[0], input.shape[0]))
		for i in np.arange(input.shape[0]):
			psistar = single_psistar(i, psistar, input)

In [147]:
args.pool = 1

In [152]:
timeit get_psistar(data,args)

1 loops, best of 3: 267 ms per loop


In [155]:
timeit max([7, 13])

The slowest run took 6.76 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 334 ns per loop


In [156]:
def qian(X,Y):
	'''A function that computes the AP-Qian similarity measure between two expression profiles.'''
	holderplus = np.zeros((len(X)+1,len(X)+1))
	holderminus = np.zeros((len(X)+1,len(X)+1))
	out = 0
	for i in np.arange(1,len(X)+1):
		for j in np.arange(1,len(X)+1):
			#don't ask. this is faster. np.max() is glacial
			h1 = holderplus[i-1,j-1] + X[i-1]*Y[j-1]
			h2 = holderminus[i-1,j-1] - X[i-1]*Y[j-1]
			if h1>0:
				holderplus[i,j] = h1
				if h1>out:
					out=h1
			if h2>0:
				holderminus[i,j] = h2
				if h2>out:
					out=h2
	return out-len(X)+1

In [157]:
timeit qian(dum1,dum1)

1000 loops, best of 3: 1.57 ms per loop


In [158]:
dum1

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [159]:
def qian(X,Y):
	'''A function that computes the AP-Qian similarity measure between two expression profiles.'''
	holderplus = np.zeros((len(X)+1,len(X)+1))
	holderminus = np.zeros((len(X)+1,len(X)+1))
	out = 0
	for i in np.arange(1,len(X)+1):
		for j in np.arange(1,len(X)+1):
			holderplus[i,j] = max([0, holderplus[i-1,j-1] + X[i-1]*Y[j-1]])
			holderminus[i,j] = max([0, holderminus[i-1,j-1] - X[i-1]*Y[j-1]])
			out = max([out, max([holderplus[i,j], holderminus[i,j]])])
	return out-len(X)+1

In [160]:
timeit qian(dum1,dum1)

100 loops, best of 3: 2.7 ms per loop


In [161]:
def qian(X,Y):
	'''A function that computes the AP-Qian similarity measure between two expression profiles.'''
	holderplus = np.zeros((len(X)+1,len(X)+1))
	holderminus = np.zeros((len(X)+1,len(X)+1))
	out = 0
	for i in np.arange(1,len(X)+1):
		for j in np.arange(1,len(X)+1):
			#don't ask. this is faster. np.max() is glacial
			h1 = holderplus[i-1,j-1] + X[i-1]*Y[j-1]
			h2 = holderminus[i-1,j-1] - X[i-1]*Y[j-1]
			if h1>0:
				holderplus[i,j] = h1
				if h1>out:
					out=h1
			if h2>0:
				holderminus[i,j] = h2
				if h2>out:
					out=h2
	return out-len(X)+1

In [163]:
timeit qian(dum1,dum1)

1000 loops, best of 3: 1.51 ms per loop


In [186]:
	psistar_base = mp.Array(ctypes.c_double, data.shape[0]*data.shape[0])
	psistar = np.ctypeslib.as_array(psistar_base.get_obj())
	psistar = psistar.reshape(data.shape[0], data.shape[0])

In [187]:
kek3 = np.copy(psistar[0,:])

In [188]:
kek3[3]=4

In [189]:
kek3

array([ 0.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.])

In [190]:
psistar

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [192]:
kek = np.random.normal(size=1000)

In [199]:
timeit np.argmax(kek)

The slowest run took 12.24 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 1.89 µs per loop


In [200]:
import math

In [205]:
float("-inf")

-inf

In [208]:
np.max(kek2,axis=1)

array([ 2.52125907,  1.99265956,  2.24608584,  2.50783448,  2.51167133,
        2.2849225 ,  2.44327732,  2.29323926,  2.79466497,  2.2910026 ,
        1.99810896,  2.60596668,  3.90755761,  2.54622656,  1.90274906,
        2.11594107,  3.22943577,  2.11377591,  2.55447125,  2.39168333,
        2.87956513,  2.49775826,  2.39703415,  2.75390132,  2.22628151,
        2.79723651,  1.93203891,  2.22157466,  2.32765344,  2.21261859,
        2.32925321,  2.11672837,  2.11949414,  3.03688221,  3.00654286,
        2.15470931,  3.03082156,  2.0917678 ,  2.98811566,  2.80307689,
        2.29723876,  2.36299173,  2.03232009,  2.66884571,  1.88345627,
        2.57166331,  2.02437624,  2.11120453,  1.95207238,  2.43080185,
        3.33980003,  2.10509567,  3.69425534,  2.10326462,  2.02423418,
        2.5397387 ,  2.86065574,  2.1460632 ,  2.77766747,  3.8138732 ,
        2.31595405,  2.33682341,  1.97656937,  1.79623752,  1.82583207,
        2.87140415,  2.2499648 ,  3.16015809,  2.20095313,  2.44

In [210]:
kek2[np.arange(100),np.argmax(kek2,axis=1)]

array([ 2.52125907,  1.99265956,  2.24608584,  2.50783448,  2.51167133,
        2.2849225 ,  2.44327732,  2.29323926,  2.79466497,  2.2910026 ,
        1.99810896,  2.60596668,  3.90755761,  2.54622656,  1.90274906,
        2.11594107,  3.22943577,  2.11377591,  2.55447125,  2.39168333,
        2.87956513,  2.49775826,  2.39703415,  2.75390132,  2.22628151,
        2.79723651,  1.93203891,  2.22157466,  2.32765344,  2.21261859,
        2.32925321,  2.11672837,  2.11949414,  3.03688221,  3.00654286,
        2.15470931,  3.03082156,  2.0917678 ,  2.98811566,  2.80307689,
        2.29723876,  2.36299173,  2.03232009,  2.66884571,  1.88345627,
        2.57166331,  2.02437624,  2.11120453,  1.95207238,  2.43080185,
        3.33980003,  2.10509567,  3.69425534,  2.10326462,  2.02423418,
        2.5397387 ,  2.86065574,  2.1460632 ,  2.77766747,  3.8138732 ,
        2.31595405,  2.33682341,  1.97656937,  1.79623752,  1.82583207,
        2.87140415,  2.2499648 ,  3.16015809,  2.20095313,  2.44

In [215]:
timeit ind = np.argmax(kek2,axis=1)

The slowest run took 8.09 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 14.5 µs per loop


In [212]:
timeit kek2[np.arange(100),ind]

The slowest run took 25.72 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 4.92 µs per loop


In [213]:
timeit np.max(kek2,axis=1)

The slowest run took 10.37 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 13.4 µs per loop


In [214]:
25*5

125

In [220]:
wololo = kek2 - np.max(kek2, axis=1)[:, None]

In [221]:
np.max(wololo, axis=1)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [222]:
psistar

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [223]:
get_psistar(data,args)

In [224]:
psistar

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [226]:
args

Namespace(conv=1000, damp=0.9, input=<_io.TextIOWrapper name='yeast_example_10.txt' mode='r' encoding='UTF-8'>, iters=1000, pool=1, s=0)

In [227]:
def single_psistar(i, psistar, data):
	'''Computes a single gene's worth of Qian similarity matrix'''
	for j in np.arange(i+1, data.shape[0]):
		val = qian(data[i,:], data[j,:])
		psistar[i,j] = val
		psistar[j,i] = val
	#no need for a return as this is done on a shared memory array

def pool_single_psistar(i):
	'''The same as above, wrapped in a parallel-happy manner'''
	#no need for a return as this is on a shared memory array
	single_psistar(i, psistar, data)

def _psistar_pool_init(_psistar, _data):
	global psistar, data
	psistar = _psistar
	data = _data

def get_psistar(data, args):
	'''Computes the AP-Qian similarity matrix'''
	t0 = time.clock()
	#set up shared memory regardless of parallelisation or not
	psistar_base = mp.Array(ctypes.c_double, data.shape[0]*data.shape[0])
	psistar = np.ctypeslib.as_array(psistar_base.get_obj())
	psistar = psistar.reshape(data.shape[0], data.shape[0])
	if args.pool > 1:
		p = mp.Pool(args.pool, _psistar_pool_init, (psistar, data))
		p.map(pool_single_psistar, np.arange(input.shape[0]))
	else:
		for i in np.arange(data.shape[0]-1):
			single_psistar(i, psistar, data)
	#now, time for self-similarity
	if args.s == 0:
		#slot in the median psistar score
		mask = np.ones(psistar.shape, dtype=bool)
		np.fill_diagonal(mask, 0)
		s1 = np.median(psistar[mask])
		np.fill_diagonal(psistar, s1)
	else:
		#just put in the provided value
		np.fill_diagonal(psistar, args.s)
	return psistar


In [228]:
psistar = get_psistar(data,args)

In [229]:
psistar

array([[ -7.8235198 ,  -9.96621859,  -8.17416598,  -9.20923155,
         -9.54997823,  -8.43338573,  -7.83480044,  -5.50684859,
         -5.85177818, -10.21868884],
       [ -9.96621859,  -7.8235198 ,  -9.14159602,  -5.92859693,
         -6.73902199,  -7.97018475,  -9.39384304,  -7.8235198 ,
         -8.90605239,  -5.53412437],
       [ -8.17416598,  -9.14159602,  -7.8235198 ,  -4.77676397,
         -7.17530533,  -6.69674174,  -9.0890633 ,  -7.97131334,
         -9.41253147,  -7.39927648],
       [ -9.20923155,  -5.92859693,  -4.77676397,  -7.8235198 ,
         -7.40891915,  -4.19955487,  -8.87038497,  -6.7263422 ,
         -7.63663341,  -2.12311401],
       [ -9.54997823,  -6.73902199,  -7.17530533,  -7.40891915,
         -7.8235198 ,  -8.39919639,  -9.98245881,  -6.91365189,
         -9.26618426,  -7.113631  ],
       [ -8.43338573,  -7.97018475,  -6.69674174,  -4.19955487,
         -8.39919639,  -7.8235198 ,  -8.32079689,  -5.98041839,
        -10.17309229,  -8.31888167],
       [ -

In [230]:
a = np.zeros(psistar.shape)
r = np.zeros(psistar.shape)

In [236]:
def rstep(a,r,psistar,args):
	old = np.copy(r)
	apsi = a+psistar
	ind1 = np.arange(apsi.shape[0])
	ind2 = np.argmax(apsi,axis=1)
	val1 = apsi[ind1, ind2]
	apsi[ind1, ind2] = float('-inf')
	val2 = np.max(apsi, axis=1)
	r = psistar - val1[:, None]
	r[ind1, ind2] = psistar[ind1, ind2] = val2
	r = args.damp*old + (1-args.damp)*r

In [237]:
timeit rstep(a,r,psistar,args)

The slowest run took 6.09 times longer than the fastest. This could mean that an intermediate result is being cached 
10000 loops, best of 3: 43.4 µs per loop


In [235]:
def single_r(i, a, r, psistar, args):
	'''Updates a single row of the R matrix'''
	for j in np.arange(r.shape[0]):
		old = r[i,j]
		holder = np.delete(a[i,:]+psistar[i,:],j)
		r[i,j] = psistar[i,j] - np.max(holder)
		#damping. screw first step skip, Matlab ain't having any of it
		r[i,j] = args.damp*old + (1-args.damp)*r[i,j]
	return r

In [260]:
def oldr(a, r, psistar, args):
    for i in np.arange(r.shape[0]):
        r = single_r(i, a, r, psistar, args)
    print(r)

In [240]:
timeit oldr(a,r,psistar,args)

100 loops, best of 3: 3.33 ms per loop


In [244]:
	apsi = a[i,:]+psistar[i,:]
	ind2 = np.argmax(apsi,axis=1)

ValueError: axis(=1) out of bounds

In [245]:
apsi

array([-9.96621859, -7.8235198 , -9.14159602, -5.92859693, -6.73902199,
       -7.97018475, -9.39384304, -7.8235198 , -8.90605239, -5.92859693])

In [250]:
def rstep(a,r,psistar,args):
	old = np.copy(r)
	apsi = a+psistar
	ind1 = np.arange(apsi.shape[0])
	ind2 = np.argmax(apsi,axis=1)
	val1 = apsi[ind1, ind2]
	apsi[ind1, ind2] = float('-inf')
	val2 = np.max(apsi, axis=1)
	r = psistar - val1[:, None]
	r[ind1, ind2] = psistar[ind1, ind2] - val2
	r = args.damp*old + (1-args.damp)*r

In [252]:
timeit rstep(a,r,psistar,args)

10000 loops, best of 3: 47.7 µs per loop


In [249]:
timeit oldr(a,r,psistar,args)

100 loops, best of 3: 3.53 ms per loop


In [253]:
def rstepi(i,a,r,psistar,args):
	old = np.copy(r[i,:])
	apsi = a[i,:]+psistar[i,:]
	ind = np.argmax(apsi)
	val1 = apsi[ind]
	apsi[ind] = float('-inf')
	val2 = np.max(apsi)
	r = psistar[i,:] - val1
	r[ind] = psistar[ind] - val2
	r = args.damp*old + (1-args.damp)*r

In [256]:
def rstepiwrap(a, r, psistar, args):
	for i in np.arange(r.shape[0]):
		r = rstepi(i, a, r, psistar, args)

In [257]:
timeit rstepiwrap(a,r,psistar,args)

ValueError: setting an array element with a sequence.

In [258]:
	old = np.copy(r[i,:])
	apsi = a[i,:]+psistar[i,:]
	ind = np.argmax(apsi)
	val1 = apsi[ind]
	apsi[ind] = float('-inf')
	val2 = np.max(apsi)

In [259]:
val2

-5.9285969348646361

In [267]:
oldr(a,r,psistar,args)

[[-0.19717416 -0.41144404 -0.23223878 -0.33574534 -0.36982001 -0.25816075
  -0.19830223  0.          0.         -0.43669107]
 [-0.40376217 -0.18949229 -0.32129991  0.         -0.08104251 -0.20415878
  -0.34652461 -0.18949229 -0.29774555  0.        ]
 [-0.14774242 -0.24448543 -0.11267781  0.         -0.04785636  0.
  -0.23923216 -0.12745716 -0.27157897 -0.07025347]
 [-0.50096767 -0.17290421 -0.05772091 -0.36239649 -0.32093643  0.
  -0.46708301 -0.25267873 -0.34370785  0.        ]
 [-0.26363263  0.         -0.02616534 -0.04952673 -0.09098679 -0.14855445
  -0.30688069  0.         -0.23525324 -0.01999791]
 [-0.24529673 -0.19897664 -0.07163234  0.         -0.2418778  -0.18431014
  -0.23403785  0.         -0.41926739 -0.23384633]
 [-0.08086447 -0.23676873 -0.20629076 -0.18442293 -0.29563031 -0.12946412
  -0.07973641  0.          0.         -0.30705127]
 [-0.00924583 -0.24091295 -0.2556923  -0.13119519 -0.14992616 -0.05660281
   0.         -0.24091295  0.         -0.06304713]
 [ 0.         -0

In [263]:
a

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [272]:
r = np.zeros(psistar.shape)

In [266]:
r

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [268]:
r

array([[-0.19717416, -0.41144404, -0.23223878, -0.33574534, -0.36982001,
        -0.25816075, -0.19830223,  0.        ,  0.        , -0.43669107],
       [-0.40376217, -0.18949229, -0.32129991,  0.        , -0.08104251,
        -0.20415878, -0.34652461, -0.18949229, -0.29774555,  0.        ],
       [-0.14774242, -0.24448543, -0.11267781,  0.        , -0.04785636,
         0.        , -0.23923216, -0.12745716, -0.27157897, -0.07025347],
       [-0.50096767, -0.17290421, -0.05772091, -0.36239649, -0.32093643,
         0.        , -0.46708301, -0.25267873, -0.34370785,  0.        ],
       [-0.26363263,  0.        , -0.02616534, -0.04952673, -0.09098679,
        -0.14855445, -0.30688069,  0.        , -0.23525324, -0.01999791],
       [-0.24529673, -0.19897664, -0.07163234,  0.        , -0.2418778 ,
        -0.18431014, -0.23403785,  0.        , -0.41926739, -0.23384633],
       [-0.08086447, -0.23676873, -0.20629076, -0.18442293, -0.29563031,
        -0.12946412, -0.07973641,  0.        

In [270]:
def rstep(a,r,psistar,args):
	old = np.copy(r)
	apsi = a+psistar
	ind1 = np.arange(apsi.shape[0])
	ind2 = np.argmax(apsi,axis=1)
	val1 = apsi[ind1, ind2]
	apsi[ind1, ind2] = float('-inf')
	val2 = np.max(apsi, axis=1)
	r = psistar - val1[:, None]
	r[ind1, ind2] = psistar[ind1, ind2] - val2
	r = args.damp*old + (1-args.damp)*r
	print(r)

In [271]:
rstep(a,r,psistar,args)

[[-0.19717416 -0.41144404 -0.23223878 -0.33574534 -0.36982001 -0.25816075
  -0.19830223  0.          0.         -0.43669107]
 [-0.40376217 -0.18949229 -0.32129991  0.         -0.08104251 -0.20415878
  -0.34652461 -0.18949229 -0.29774555  0.        ]
 [-0.14774242 -0.24448543 -0.11267781  0.         -0.04785636  0.
  -0.23923216 -0.12745716 -0.27157897 -0.07025347]
 [-0.50096767 -0.17290421 -0.05772091 -0.36239649 -0.32093643  0.
  -0.46708301 -0.25267873 -0.34370785  0.        ]
 [-0.26363263  0.         -0.02616534 -0.04952673 -0.09098679 -0.14855445
  -0.30688069  0.         -0.23525324 -0.01999791]
 [-0.24529673 -0.19897664 -0.07163234  0.         -0.2418778  -0.18431014
  -0.23403785  0.         -0.41926739 -0.23384633]
 [-0.08086447 -0.23676873 -0.20629076 -0.18442293 -0.29563031 -0.12946412
  -0.07973641  0.          0.         -0.30705127]
 [-0.00924583 -0.24091295 -0.2556923  -0.13119519 -0.14992616 -0.05660281
   0.         -0.24091295  0.         -0.06304713]
 [ 0.         -0

In [289]:
def rstepi(i,a,r,psistar,args):
	old = np.copy(r[i,:])
	apsi = a[i,:]+psistar[i,:]
	ind = np.argmax(apsi)
	val1 = apsi[ind]
	apsi[ind] = float('-inf')
	val2 = np.max(apsi)
	r[i,:] = psistar[i,:] - val1
	r[i,ind] = psistar[i,ind] - val2
	r[i,:] = args.damp*old + (1-args.damp)*r[i,:]

In [293]:
r = np.zeros(psistar.shape)

In [294]:
def rstepiwrap(a, r, psistar, args):
	for i in np.arange(r.shape[0]):
		r = rstepi(i, a, r, psistar, args)

In [295]:
timeit rstepiwrap(a,r,psistar,args)

TypeError: 'NoneType' object is not subscriptable

3

In [282]:
	old = np.copy(r[i,:])
	apsi = a[i,:]+psistar[i,:]
	ind = np.argmax(apsi)
	val1 = apsi[ind]
	apsi[ind] = float('-inf')
	val2 = np.max(apsi)

In [283]:
old

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [284]:
i

1

In [285]:
apsi

array([-9.96621859, -7.8235198 , -9.14159602,        -inf, -6.73902199,
       -7.97018475, -9.39384304, -7.8235198 , -8.90605239, -5.92859693])

In [287]:
psistar[i,:]

array([-9.96621859, -7.8235198 , -9.14159602, -5.92859693, -6.73902199,
       -7.97018475, -9.39384304, -7.8235198 , -8.90605239, -5.92859693])

In [288]:
val2

-5.9285969348646361

In [296]:
rstepiwrap(a,r,psistar,args)

TypeError: 'NoneType' object is not subscriptable

In [297]:
def rstepi(i,a,r,psistar,args):
	print(r)
	print(i)
	old = np.copy(r[i,:])
	apsi = a[i,:]+psistar[i,:]
	ind = np.argmax(apsi)
	val1 = apsi[ind]
	apsi[ind] = float('-inf')
	val2 = np.max(apsi)
	r[i,:] = psistar[i,:] - val1
	r[i,ind] = psistar[i,ind] - val2
	r[i,:] = args.damp*old + (1-args.damp)*r[i,:]

In [298]:
rstepiwrap(a,r,psistar,args)

[[-0.37463091 -0.78174368 -0.44125368 -0.63791614 -0.70265801 -0.49050543
  -0.37677423  0.          0.         -0.82971303]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.   

TypeError: 'NoneType' object is not subscriptable

In [306]:
def rstepi(i,a,r,psistar,args):
	print(i)
	old = np.copy(r[i,:])
	apsi = a[i,:]+psistar[i,:]
	ind = np.argmax(apsi)
	val1 = apsi[ind]
	apsi[ind] = float('-inf')
	val2 = np.max(apsi)
	r[i,:] = psistar[i,:] - val1
	r[i,ind] = psistar[i,ind] - val2
	r[i,:] = args.damp*old + (1-args.damp)*r[i,:]

In [307]:
r = np.zeros(psistar.shape)

In [308]:
rstepi(2,a,r,psistar,args)

2


In [309]:
rstepiwrap(a,r,psistar,args)

0
1


TypeError: 'NoneType' object is not subscriptable

In [310]:
r

array([[-0.19717416, -0.41144404, -0.23223878, -0.33574534, -0.36982001,
        -0.25816075, -0.19830223,  0.        ,  0.        , -0.43669107],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.14774242, -0.24448543, -0.11267781,  0.        , -0.04785636,
         0.        , -0.23923216, -0.12745716, -0.27157897, -0.07025347],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        

In [311]:
r = np.zeros(psistar.shape)

In [312]:
psistar

array([[ -7.8235198 ,  -9.96621859,  -8.17416598,  -9.20923155,
         -9.54997823,  -8.43338573,  -7.83480044,  -5.85177818,
         -5.85177818, -10.21868884],
       [ -9.96621859,  -7.8235198 ,  -9.14159602,  -5.92859693,
         -6.73902199,  -7.97018475,  -9.39384304,  -7.8235198 ,
         -8.90605239,  -5.92859693],
       [ -8.17416598,  -9.14159602,  -7.8235198 ,  -6.69674174,
         -7.17530533,  -6.69674174,  -9.0890633 ,  -7.97131334,
         -9.41253147,  -7.39927648],
       [ -9.20923155,  -5.92859693,  -4.77676397,  -7.8235198 ,
         -7.40891915,  -4.19955487,  -8.87038497,  -6.7263422 ,
         -7.63663341,  -4.19955487],
       [ -9.54997823,  -6.91365189,  -7.17530533,  -7.40891915,
         -7.8235198 ,  -8.39919639,  -9.98245881,  -6.91365189,
         -9.26618426,  -7.113631  ],
       [ -8.43338573,  -7.97018475,  -6.69674174,  -5.98041839,
         -8.39919639,  -7.8235198 ,  -8.32079689,  -5.98041839,
        -10.17309229,  -8.31888167],
       [ -

In [313]:
rstep(a,r,psistar,args)

[[-0.19717416 -0.41144404 -0.23223878 -0.33574534 -0.36982001 -0.25816075
  -0.19830223  0.          0.         -0.43669107]
 [-0.40376217 -0.18949229 -0.32129991  0.         -0.08104251 -0.20415878
  -0.34652461 -0.18949229 -0.29774555  0.        ]
 [-0.14774242 -0.24448543 -0.11267781  0.         -0.04785636  0.
  -0.23923216 -0.12745716 -0.27157897 -0.07025347]
 [-0.50096767 -0.17290421 -0.05772091 -0.36239649 -0.32093643  0.
  -0.46708301 -0.25267873 -0.34370785  0.        ]
 [-0.26363263  0.         -0.02616534 -0.04952673 -0.09098679 -0.14855445
  -0.30688069  0.         -0.23525324 -0.01999791]
 [-0.24529673 -0.19897664 -0.07163234  0.         -0.2418778  -0.18431014
  -0.23403785  0.         -0.41926739 -0.23384633]
 [-0.08086447 -0.23676873 -0.20629076 -0.18442293 -0.29563031 -0.12946412
  -0.07973641  0.          0.         -0.30705127]
 [-0.00924583 -0.24091295 -0.2556923  -0.13119519 -0.14992616 -0.05660281
   0.         -0.24091295  0.         -0.06304713]
 [ 0.         -0

In [314]:
r = np.zeros(psistar.shape)

In [315]:
r

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [316]:
oldr(a,r,psistar,args)

[[-0.19717416 -0.41144404 -0.23223878 -0.33574534 -0.36982001 -0.25816075
  -0.19830223  0.          0.         -0.43669107]
 [-0.40376217 -0.18949229 -0.32129991  0.         -0.08104251 -0.20415878
  -0.34652461 -0.18949229 -0.29774555  0.        ]
 [-0.14774242 -0.24448543 -0.11267781  0.         -0.04785636  0.
  -0.23923216 -0.12745716 -0.27157897 -0.07025347]
 [-0.50096767 -0.17290421 -0.05772091 -0.36239649 -0.32093643  0.
  -0.46708301 -0.25267873 -0.34370785  0.        ]
 [-0.26363263  0.         -0.02616534 -0.04952673 -0.09098679 -0.14855445
  -0.30688069  0.         -0.23525324 -0.01999791]
 [-0.24529673 -0.19897664 -0.07163234  0.         -0.2418778  -0.18431014
  -0.23403785  0.         -0.41926739 -0.23384633]
 [-0.08086447 -0.23676873 -0.20629076 -0.18442293 -0.29563031 -0.12946412
  -0.07973641  0.          0.         -0.30705127]
 [-0.00924583 -0.24091295 -0.2556923  -0.13119519 -0.14992616 -0.05660281
   0.         -0.24091295  0.         -0.06304713]
 [ 0.         -0

In [317]:
r

array([[-0.19717416, -0.41144404, -0.23223878, -0.33574534, -0.36982001,
        -0.25816075, -0.19830223,  0.        ,  0.        , -0.43669107],
       [-0.40376217, -0.18949229, -0.32129991,  0.        , -0.08104251,
        -0.20415878, -0.34652461, -0.18949229, -0.29774555,  0.        ],
       [-0.14774242, -0.24448543, -0.11267781,  0.        , -0.04785636,
         0.        , -0.23923216, -0.12745716, -0.27157897, -0.07025347],
       [-0.50096767, -0.17290421, -0.05772091, -0.36239649, -0.32093643,
         0.        , -0.46708301, -0.25267873, -0.34370785,  0.        ],
       [-0.26363263,  0.        , -0.02616534, -0.04952673, -0.09098679,
        -0.14855445, -0.30688069,  0.        , -0.23525324, -0.01999791],
       [-0.24529673, -0.19897664, -0.07163234,  0.        , -0.2418778 ,
        -0.18431014, -0.23403785,  0.        , -0.41926739, -0.23384633],
       [-0.08086447, -0.23676873, -0.20629076, -0.18442293, -0.29563031,
        -0.12946412, -0.07973641,  0.        

In [318]:
r = np.zeros(psistar.shape)

In [319]:
def rstepiwrap(a, r, psistar, args):
	for i in np.arange(r.shape[0]):
		rstepi(i, a, r, psistar, args)

In [320]:
timeit rstepiwrap(a,r,psistar,args)

0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9


In [321]:
def rstepi(i,a,r,psistar,args):
	old = np.copy(r[i,:])
	apsi = a[i,:]+psistar[i,:]
	ind = np.argmax(apsi)
	val1 = apsi[ind]
	apsi[ind] = float('-inf')
	val2 = np.max(apsi)
	r[i,:] = psistar[i,:] - val1
	r[i,ind] = psistar[i,ind] - val2
	r[i,:] = args.damp*old + (1-args.damp)*r[i,:]

In [322]:
timeit rstepiwrap(a,r,psistar,args)

1000 loops, best of 3: 354 µs per loop


In [323]:
r = np.zeros(psistar.shape)

In [324]:
psistar

array([[ -7.8235198 ,  -9.96621859,  -8.17416598,  -9.20923155,
         -9.54997823,  -8.43338573,  -7.83480044,  -5.85177818,
         -5.85177818, -10.21868884],
       [ -9.96621859,  -7.8235198 ,  -9.14159602,  -5.92859693,
         -6.73902199,  -7.97018475,  -9.39384304,  -7.8235198 ,
         -8.90605239,  -5.92859693],
       [ -8.17416598,  -9.14159602,  -7.8235198 ,  -6.69674174,
         -7.17530533,  -6.69674174,  -9.0890633 ,  -7.97131334,
         -9.41253147,  -7.39927648],
       [ -9.20923155,  -5.92859693,  -4.77676397,  -7.8235198 ,
         -7.40891915,  -4.19955487,  -8.87038497,  -6.7263422 ,
         -7.63663341,  -4.19955487],
       [ -9.54997823,  -6.91365189,  -7.17530533,  -7.40891915,
         -7.8235198 ,  -8.39919639,  -9.98245881,  -6.91365189,
         -9.26618426,  -7.113631  ],
       [ -8.43338573,  -7.97018475,  -6.69674174,  -5.98041839,
         -8.39919639,  -7.8235198 ,  -8.32079689,  -5.98041839,
        -10.17309229,  -8.31888167],
       [ -

In [325]:
args.damp

0.9

In [326]:
apsi = a+psistar

In [327]:
r

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [328]:
	ind1 = np.arange(apsi.shape[0])
	ind2 = np.argmax(apsi,axis=1)

In [329]:
ind2

array([7, 3, 3, 5, 1, 3, 7, 6, 0, 1])

In [330]:
apsi

array([[ -7.8235198 ,  -9.96621859,  -8.17416598,  -9.20923155,
         -9.54997823,  -8.43338573,  -7.83480044,  -5.85177818,
         -5.85177818, -10.21868884],
       [ -9.96621859,  -7.8235198 ,  -9.14159602,  -5.92859693,
         -6.73902199,  -7.97018475,  -9.39384304,  -7.8235198 ,
         -8.90605239,  -5.92859693],
       [ -8.17416598,  -9.14159602,  -7.8235198 ,  -6.69674174,
         -7.17530533,  -6.69674174,  -9.0890633 ,  -7.97131334,
         -9.41253147,  -7.39927648],
       [ -9.20923155,  -5.92859693,  -4.77676397,  -7.8235198 ,
         -7.40891915,  -4.19955487,  -8.87038497,  -6.7263422 ,
         -7.63663341,  -4.19955487],
       [ -9.54997823,  -6.91365189,  -7.17530533,  -7.40891915,
         -7.8235198 ,  -8.39919639,  -9.98245881,  -6.91365189,
         -9.26618426,  -7.113631  ],
       [ -8.43338573,  -7.97018475,  -6.69674174,  -5.98041839,
         -8.39919639,  -7.8235198 ,  -8.32079689,  -5.98041839,
        -10.17309229,  -8.31888167],
       [ -

In [339]:
apsi[np.arange(10),np.argmax(apsi,axis=0)]

array([-5.85177818, -5.92859693, -6.69674174, -4.19955487, -6.91365189,
       -5.98041839, -7.02615571, -5.50684859, -5.85177818, -5.53412437])

In [342]:
temp1 = np.argmax(apsi,axis=1)

In [343]:
temp1

array([7, 3, 3, 5, 1, 3, 7, 6, 0, 1])

In [351]:
apsi[1,temp[1]]

-5.9285969348646361

In [352]:
kek = np.ones((2,5))

In [354]:
np.max(kek,axis=1)

array([ 1.,  1.])

In [355]:
def rstep(a,r,psistar,args):
	'''A full vectorised single R step'''
	old = np.copy(r)
	apsi = a+psistar
	ind1 = np.arange(apsi.shape[0])
	ind2 = np.argmax(apsi,axis=1)
	val1 = apsi[ind1, ind2]
	apsi[ind1, ind2] = float('-inf')
	val2 = np.max(apsi, axis=1)
	r = psistar - val1[:, None]
	r[ind1, ind2] = psistar[ind1, ind2] - val2
	r = args.damp*old + (1-args.damp)*r
	print(r)

In [357]:
rstep(a,r,psistar,args)

[[-0.19717416 -0.41144404 -0.23223878 -0.33574534 -0.36982001 -0.25816075
  -0.19830223  0.          0.         -0.43669107]
 [-0.40376217 -0.18949229 -0.32129991  0.         -0.08104251 -0.20415878
  -0.34652461 -0.18949229 -0.29774555  0.        ]
 [-0.14774242 -0.24448543 -0.11267781  0.         -0.04785636  0.
  -0.23923216 -0.12745716 -0.27157897 -0.07025347]
 [-0.50096767 -0.17290421 -0.05772091 -0.36239649 -0.32093643  0.
  -0.46708301 -0.25267873 -0.34370785  0.        ]
 [-0.26363263  0.         -0.02616534 -0.04952673 -0.09098679 -0.14855445
  -0.30688069  0.         -0.23525324 -0.01999791]
 [-0.24529673 -0.19897664 -0.07163234  0.         -0.2418778  -0.18431014
  -0.23403785  0.         -0.41926739 -0.23384633]
 [-0.08086447 -0.23676873 -0.20629076 -0.18442293 -0.29563031 -0.12946412
  -0.07973641  0.          0.         -0.30705127]
 [-0.00924583 -0.24091295 -0.2556923  -0.13119519 -0.14992616 -0.05660281
   0.         -0.24091295  0.         -0.06304713]
 [ 0.         -0

In [361]:
r = np.zeros(psistar.shape[0])

In [362]:
	old = np.copy(r)
	apsi = a+psistar
	ind1 = np.arange(apsi.shape[0])
	ind2 = np.argmax(apsi,axis=0)
	val1 = apsi[ind1, ind2]
	apsi[ind1, ind2] = float('-inf')
	val2 = np.max(apsi, axis=1)
	r = psistar - val1[:, None]
	r[ind1, ind2] = psistar[ind1, ind2] - val2
	r = args.damp*old + (1-args.damp)*r

In [364]:
apsi

array([[ -7.8235198 ,  -9.96621859,  -8.17416598,  -9.20923155,
         -9.54997823,  -8.43338573,  -7.83480044,         -inf,
         -5.85177818, -10.21868884],
       [ -9.96621859,  -7.8235198 ,  -9.14159602,  -5.92859693,
         -6.73902199,  -7.97018475,  -9.39384304,  -7.8235198 ,
         -8.90605239,         -inf],
       [ -8.17416598,  -9.14159602,  -7.8235198 ,         -inf,
         -7.17530533,  -6.69674174,  -9.0890633 ,  -7.97131334,
         -9.41253147,  -7.39927648],
       [ -9.20923155,  -5.92859693,  -4.77676397,  -7.8235198 ,
         -7.40891915,  -4.19955487,  -8.87038497,  -6.7263422 ,
         -7.63663341,         -inf],
       [ -9.54997823,         -inf,  -7.17530533,  -7.40891915,
         -7.8235198 ,  -8.39919639,  -9.98245881,  -6.91365189,
         -9.26618426,  -7.113631  ],
       [ -8.43338573,  -7.97018475,  -6.69674174,         -inf,
         -8.39919639,  -7.8235198 ,  -8.32079689,  -5.98041839,
        -10.17309229,  -8.31888167],
       [ -

In [366]:
apsi = a+psistar

In [367]:
np.max(apsi)

-4.1995548683820108

In [368]:
apsi

array([[ -7.8235198 ,  -9.96621859,  -8.17416598,  -9.20923155,
         -9.54997823,  -8.43338573,  -7.83480044,  -5.85177818,
         -5.85177818, -10.21868884],
       [ -9.96621859,  -7.8235198 ,  -9.14159602,  -5.92859693,
         -6.73902199,  -7.97018475,  -9.39384304,  -7.8235198 ,
         -8.90605239,  -5.92859693],
       [ -8.17416598,  -9.14159602,  -7.8235198 ,  -6.69674174,
         -7.17530533,  -6.69674174,  -9.0890633 ,  -7.97131334,
         -9.41253147,  -7.39927648],
       [ -9.20923155,  -5.92859693,  -4.77676397,  -7.8235198 ,
         -7.40891915,  -4.19955487,  -8.87038497,  -6.7263422 ,
         -7.63663341,  -4.19955487],
       [ -9.54997823,  -6.91365189,  -7.17530533,  -7.40891915,
         -7.8235198 ,  -8.39919639,  -9.98245881,  -6.91365189,
         -9.26618426,  -7.113631  ],
       [ -8.43338573,  -7.97018475,  -6.69674174,  -5.98041839,
         -8.39919639,  -7.8235198 ,  -8.32079689,  -5.98041839,
        -10.17309229,  -8.31888167],
       [ -

In [369]:
data

array([[ -4.58531963e-01,   4.21290789e-01,   4.42238949e-02,
          2.53705503e-01,   1.69912860e-01,  -1.08697679e+00,
          1.69912860e-01,   7.98357682e-01,  -9.19391500e-01,
         -8.35598857e-01,   5.05083432e-01,   8.82150325e-01,
         -1.08697679e+00,  -1.25456207e+00,   9.24046647e-01,
          2.43231422e+00,  -1.50594000e+00,   5.46979753e-01],
       [ -4.00214640e-01,  -2.04269552e+00,   2.91356258e-01,
         -9.18892814e-01,  -1.14941645e+00,  -1.66809462e+00,
          3.20171712e-01,   6.08326253e-01,   2.62540804e-01,
          1.64568260e+00,  -4.00214640e-01,   8.38849886e-01,
          3.20171712e-02,   1.18463534e-01,   1.47278988e-01,
          1.58805169e+00,  -2.27321916e-01,   9.54111703e-01],
       [ -6.16524428e-02,  -1.60856828e+00,  -3.64309889e-01,
          4.09148030e-01,  -9.52810480e-02,   6.44548266e-01,
          4.42776635e-01,   2.56137876e+00,   2.41005004e-01,
          1.40119188e-01,  -1.96166864e-01,   1.01446292e+00,
      

In [370]:
qian(data[3,:],data[9,:])

-2.1231140100078001

In [371]:
def qian(X,Y):
	'''A function that computes the AP-Qian similarity measure between two expression profiles.'''
	holderplus = np.zeros((len(X)+1,len(X)+1))
	holderminus = np.zeros((len(X)+1,len(X)+1))
	out = 0
	for i in np.arange(1,len(X)+1):
		for j in np.arange(1,len(X)+1):
			#don't ask. this is faster. np.max() is glacial
			h1 = holderplus[i-1,j-1] + X[i-1]*Y[j-1]
			h2 = holderminus[i-1,j-1] - X[i-1]*Y[j-1]
			if h1>0:
				holderplus[i,j] = h1
				if h1>out:
					out=h1
			if h2>0:
				holderminus[i,j] = h2
				if h2>out:
					out=h2
	return out-len(X)+1

def single_psistar(i, psistar, data):
	'''Computes a single gene's worth of Qian similarity matrix'''
	for j in np.arange(i+1, data.shape[0]):
		val = qian(data[i,:], data[j,:])
		psistar[i,j] = val
		psistar[j,i] = val
	#no need for a return as this is done on a shared memory array

def pool_single_psistar(i):
	'''The same as above, wrapped in a parallel-happy manner'''
	#no need for a return as this is on a shared memory array
	single_psistar(i, psistar, data)

def _psistar_pool_init(_psistar, _data):
	global psistar, data
	psistar = _psistar
	data = _data

def get_psistar(data, args):
	'''Computes the AP-Qian similarity matrix'''
	#set up shared memory regardless of parallelisation or not
	psistar_base = mp.Array(ctypes.c_double, data.shape[0]*data.shape[0])
	psistar = np.ctypeslib.as_array(psistar_base.get_obj())
	psistar = psistar.reshape(data.shape[0], data.shape[0])
	if args.pool > 1:
		p = mp.Pool(args.pool, _psistar_pool_init, (psistar, data))
		p.map(pool_single_psistar, np.arange(data.shape[0]))
	else:
		for i in np.arange(data.shape[0]-1):
			single_psistar(i, psistar, data)
	#now, time for self-similarity
	if args.s == 0:
		#slot in the median psistar score
		mask = np.ones(psistar.shape, dtype=bool)
		np.fill_diagonal(mask, 0)
		s1 = np.median(psistar[mask])
		np.fill_diagonal(psistar, s1)
	else:
		#just put in the provided value
		np.fill_diagonal(psistar, args.s)
	return psistar


In [373]:
psistar = get_psistar(data,args)

In [374]:
psistar

array([[ -7.8235198 ,  -9.96621859,  -8.17416598,  -9.20923155,
         -9.54997823,  -8.43338573,  -7.83480044,  -5.50684859,
         -5.85177818, -10.21868884],
       [ -9.96621859,  -7.8235198 ,  -9.14159602,  -5.92859693,
         -6.73902199,  -7.97018475,  -9.39384304,  -7.8235198 ,
         -8.90605239,  -5.53412437],
       [ -8.17416598,  -9.14159602,  -7.8235198 ,  -4.77676397,
         -7.17530533,  -6.69674174,  -9.0890633 ,  -7.97131334,
         -9.41253147,  -7.39927648],
       [ -9.20923155,  -5.92859693,  -4.77676397,  -7.8235198 ,
         -7.40891915,  -4.19955487,  -8.87038497,  -6.7263422 ,
         -7.63663341,  -2.12311401],
       [ -9.54997823,  -6.73902199,  -7.17530533,  -7.40891915,
         -7.8235198 ,  -8.39919639,  -9.98245881,  -6.91365189,
         -9.26618426,  -7.113631  ],
       [ -8.43338573,  -7.97018475,  -6.69674174,  -4.19955487,
         -8.39919639,  -7.8235198 ,  -8.32079689,  -5.98041839,
        -10.17309229,  -8.31888167],
       [ -

In [375]:
r = np.zeros(psistar.shape)

In [376]:
psistar

array([[ -7.8235198 ,  -9.96621859,  -8.17416598,  -9.20923155,
         -9.54997823,  -8.43338573,  -7.83480044,  -5.50684859,
         -5.85177818, -10.21868884],
       [ -9.96621859,  -7.8235198 ,  -9.14159602,  -5.92859693,
         -6.73902199,  -7.97018475,  -9.39384304,  -7.8235198 ,
         -8.90605239,  -5.53412437],
       [ -8.17416598,  -9.14159602,  -7.8235198 ,  -4.77676397,
         -7.17530533,  -6.69674174,  -9.0890633 ,  -7.97131334,
         -9.41253147,  -7.39927648],
       [ -9.20923155,  -5.92859693,  -4.77676397,  -7.8235198 ,
         -7.40891915,  -4.19955487,  -8.87038497,  -6.7263422 ,
         -7.63663341,  -2.12311401],
       [ -9.54997823,  -6.73902199,  -7.17530533,  -7.40891915,
         -7.8235198 ,  -8.39919639,  -9.98245881,  -6.91365189,
         -9.26618426,  -7.113631  ],
       [ -8.43338573,  -7.97018475,  -6.69674174,  -4.19955487,
         -8.39919639,  -7.8235198 ,  -8.32079689,  -5.98041839,
        -10.17309229,  -8.31888167],
       [ -

In [377]:
a

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [384]:
def rstep(a,r,psistar,args):
    '''A full vectorised single R step'''
    old = np.copy(r)
    apsi = a+psistar
    ind1 = np.arange(apsi.shape[0])
    ind2 = np.argmax(apsi,axis=1)
    val1 = apsi[ind1, ind2]
    apsi[ind1, ind2] = float('-inf')
    val2 = np.max(apsi, axis=1)
    r = psistar - val1[:, None]
    r[ind1, ind2] = psistar[ind1, ind2] - val2
    r = args.damp*old + (1-args.damp)*r
    return r

In [385]:
r = rstep(a,r,psistar,args)

In [383]:
r-np.arange(10)[None,:]

array([[ 0., -1., -2., -3., -4., -5., -6., -7., -8., -9.],
       [ 0., -1., -2., -3., -4., -5., -6., -7., -8., -9.],
       [ 0., -1., -2., -3., -4., -5., -6., -7., -8., -9.],
       [ 0., -1., -2., -3., -4., -5., -6., -7., -8., -9.],
       [ 0., -1., -2., -3., -4., -5., -6., -7., -8., -9.],
       [ 0., -1., -2., -3., -4., -5., -6., -7., -8., -9.],
       [ 0., -1., -2., -3., -4., -5., -6., -7., -8., -9.],
       [ 0., -1., -2., -3., -4., -5., -6., -7., -8., -9.],
       [ 0., -1., -2., -3., -4., -5., -6., -7., -8., -9.],
       [ 0., -1., -2., -3., -4., -5., -6., -7., -8., -9.]])

In [386]:
r

array([[-0.23166712, -0.445937  , -0.26673174, -0.3702383 , -0.40431296,
        -0.29265371, -0.23279519,  0.03449296, -0.03449296, -0.47118403],
       [-0.44320942, -0.22893954, -0.36074717, -0.03944726, -0.12048976,
        -0.24360604, -0.38597187, -0.22893954, -0.3371928 ,  0.03944726],
       [-0.3397402 , -0.43648321, -0.30467558,  0.19199778, -0.23985414,
        -0.19199778, -0.43122993, -0.31945494, -0.46357675, -0.26225125],
       [-0.70861175, -0.38054829, -0.265365  , -0.57004058, -0.52858051,
        -0.20764409, -0.6747271 , -0.46032282, -0.55135194,  0.20764409],
       [-0.28109562,  0.01746299, -0.04362833, -0.06698972, -0.10844978,
        -0.16601744, -0.32434368, -0.01746299, -0.25271623, -0.0374609 ],
       [-0.42338309, -0.37706299, -0.24971869,  0.17808635, -0.41996415,
        -0.36239649, -0.4121242 , -0.17808635, -0.59735374, -0.41193268],
       [-0.25643389, -0.41233815, -0.38186017, -0.35999234, -0.47119972,
        -0.30503353, -0.25530582,  0.17556941

In [402]:
def astep(a,r,psistar,args):
	'''A full vectorised single A step'''
	old = np.copy(a)
	rp = np.copy(r)
	rp[rp<0] = 0
	np.fill_diagonal(rp, r.diagonal())
	a = np.sum(rp,axis=0)[None,:] - rp
	ad = a.diagonal()
	a[a>0] = 0
	np.fill_diagonal(a, ad)
	a = args.damp*old + (1-args.damp)*a
	print(a)

In [393]:
a1=astep(a,r,psistar,args)

In [395]:
a1[:,1]

array([-0.02114766,  0.        , -0.02114766, -0.02114766, -0.02289395,
       -0.02114766, -0.02114766, -0.02114766, -0.02114766, -0.02114766])

In [397]:
r

array([[-0.23166712, -0.445937  , -0.26673174, -0.3702383 , -0.40431296,
        -0.29265371, -0.23279519,  0.03449296, -0.03449296, -0.47118403],
       [-0.44320942, -0.22893954, -0.36074717, -0.03944726, -0.12048976,
        -0.24360604, -0.38597187, -0.22893954, -0.3371928 ,  0.03944726],
       [-0.3397402 , -0.43648321, -0.30467558,  0.19199778, -0.23985414,
        -0.19199778, -0.43122993, -0.31945494, -0.46357675, -0.26225125],
       [-0.70861175, -0.38054829, -0.265365  , -0.57004058, -0.52858051,
        -0.20764409, -0.6747271 , -0.46032282, -0.55135194,  0.20764409],
       [-0.28109562,  0.01746299, -0.04362833, -0.06698972, -0.10844978,
        -0.16601744, -0.32434368, -0.01746299, -0.25271623, -0.0374609 ],
       [-0.42338309, -0.37706299, -0.24971869,  0.17808635, -0.41996415,
        -0.36239649, -0.4121242 , -0.17808635, -0.59735374, -0.41193268],
       [-0.25643389, -0.41233815, -0.38186017, -0.35999234, -0.47119972,
        -0.30503353, -0.25530582,  0.17556941

In [398]:
r.diagonal()

array([-0.23166712, -0.22893954, -0.30467558, -0.57004058, -0.10844978,
       -0.36239649, -0.25530582, -0.25530582, -0.24091295, -0.57004058])

In [399]:
a1.diagonal()

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [400]:
	old = np.copy(a)
	rp = np.copy(r)
	rp[rp<0] = 0
	np.fill_diagonal(rp, r.diagonal())
	a = np.sum(rp,axis=0)[None,:] - rp

In [401]:
a.diagonal()

array([ 0.        ,  0.01746299,  0.        ,  0.71118517,  0.        ,
        0.        ,  0.01439287,  0.25380116,  0.        ,  0.24709134])

In [417]:
a = np.zeros(psistar.shape)

In [418]:
a

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [419]:
	old = np.copy(a)
	rp = np.copy(r)
	rp[rp<0] = 0
	np.fill_diagonal(rp, r.diagonal())
	a = np.sum(rp,axis=0)[None,:] - rp

In [420]:
a.diagonal()

array([ 0.        ,  0.01746299,  0.        ,  0.71118517,  0.        ,
        0.        ,  0.01439287,  0.25380116,  0.        ,  0.24709134])

In [421]:
	ad = a.diagonal()

In [422]:
ad

array([ 0.        ,  0.01746299,  0.        ,  0.71118517,  0.        ,
        0.        ,  0.01439287,  0.25380116,  0.        ,  0.24709134])

In [423]:
	a[a>0] = 0

In [424]:
a

array([[ 0.        , -0.21147655, -0.30467558,  0.        , -0.10844978,
        -0.36239649, -0.24091295, -0.03599762, -0.24091295, -0.32294924],
       [-0.23166712,  0.        , -0.30467558,  0.        , -0.10844978,
        -0.36239649, -0.24091295, -0.00150466, -0.24091295, -0.36239649],
       [-0.23166712, -0.21147655,  0.        , -0.05085319, -0.10844978,
        -0.36239649, -0.24091295, -0.00150466, -0.24091295, -0.32294924],
       [-0.23166712, -0.21147655, -0.30467558,  0.        , -0.10844978,
        -0.36239649, -0.24091295, -0.00150466, -0.24091295, -0.53059332],
       [-0.23166712, -0.22893954, -0.30467558,  0.        ,  0.        ,
        -0.36239649, -0.24091295, -0.00150466, -0.24091295, -0.32294924],
       [-0.23166712, -0.21147655, -0.30467558, -0.03694177, -0.10844978,
         0.        , -0.24091295, -0.00150466, -0.24091295, -0.32294924],
       [-0.23166712, -0.21147655, -0.30467558,  0.        , -0.10844978,
        -0.36239649,  0.        , -0.17707408

In [425]:
a.diagonal() = ad

SyntaxError: can't assign to function call (<ipython-input-425-229595f8e879>, line 1)

In [426]:
a

array([[ 0.        , -0.21147655, -0.30467558,  0.        , -0.10844978,
        -0.36239649, -0.24091295, -0.03599762, -0.24091295, -0.32294924],
       [-0.23166712,  0.        , -0.30467558,  0.        , -0.10844978,
        -0.36239649, -0.24091295, -0.00150466, -0.24091295, -0.36239649],
       [-0.23166712, -0.21147655,  0.        , -0.05085319, -0.10844978,
        -0.36239649, -0.24091295, -0.00150466, -0.24091295, -0.32294924],
       [-0.23166712, -0.21147655, -0.30467558,  0.        , -0.10844978,
        -0.36239649, -0.24091295, -0.00150466, -0.24091295, -0.53059332],
       [-0.23166712, -0.22893954, -0.30467558,  0.        ,  0.        ,
        -0.36239649, -0.24091295, -0.00150466, -0.24091295, -0.32294924],
       [-0.23166712, -0.21147655, -0.30467558, -0.03694177, -0.10844978,
         0.        , -0.24091295, -0.00150466, -0.24091295, -0.32294924],
       [-0.23166712, -0.21147655, -0.30467558,  0.        , -0.10844978,
        -0.36239649,  0.        , -0.17707408

In [427]:
a.diagonal()

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [428]:
ad

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [429]:
def astep(a,r,psistar,args):
	'''A full vectorised single A step'''
	old = np.copy(a)
	rp = np.copy(r)
	rp[rp<0] = 0
	np.fill_diagonal(rp, r.diagonal())
	a = np.sum(rp,axis=0)[None,:] - rp
	ad = np.copy(a.diagonal())
	a[a>0] = 0
	np.fill_diagonal(a, ad)
	a = args.damp*old + (1-args.damp)*a
	print(a)

In [430]:
a = np.zeros(psistar.shape)

In [431]:
astep(a,r,psistar,args)

[[ 0.         -0.02114766 -0.03046756  0.         -0.01084498 -0.03623965
  -0.02409129 -0.00359976 -0.02409129 -0.03229492]
 [-0.02316671  0.0017463  -0.03046756  0.         -0.01084498 -0.03623965
  -0.02409129 -0.00015047 -0.02409129 -0.03623965]
 [-0.02316671 -0.02114766  0.         -0.00508532 -0.01084498 -0.03623965
  -0.02409129 -0.00015047 -0.02409129 -0.03229492]
 [-0.02316671 -0.02114766 -0.03046756  0.07111852 -0.01084498 -0.03623965
  -0.02409129 -0.00015047 -0.02409129 -0.05305933]
 [-0.02316671 -0.02289395 -0.03046756  0.          0.         -0.03623965
  -0.02409129 -0.00015047 -0.02409129 -0.03229492]
 [-0.02316671 -0.02114766 -0.03046756 -0.00369418 -0.01084498  0.
  -0.02409129 -0.00015047 -0.02409129 -0.03229492]
 [-0.02316671 -0.02114766 -0.03046756  0.         -0.01084498 -0.03623965
   0.00143929 -0.01770741 -0.02409129 -0.03229492]
 [-0.02316671 -0.02114766 -0.03046756  0.         -0.01084498 -0.03623965
  -0.02553058  0.02538012 -0.02409129 -0.03229492]
 [-0.023

In [432]:
def astep(a,r,psistar,args):
	'''A full vectorised single A step'''
	old = np.copy(a)
	rp = np.copy(r)
	rp[rp<0] = 0
	np.fill_diagonal(rp, r.diagonal())
	a = np.sum(rp,axis=0)[None,:] - rp
	ad = np.copy(a.diagonal())
	a[a>0] = 0
	np.fill_diagonal(a, ad)
	a = args.damp*old + (1-args.damp)*a

In [434]:
timeit astep(a,r,psistar,args)

10000 loops, best of 3: 52.7 µs per loop
